This notebook is for import

In [1]:
from __future__ import print_function
from __future__ import division
from sklearn import cross_validation
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
import seaborn as sns
import itertools
from scipy.stats import gaussian_kde
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.core import Dense, Dropout, Activation
from keras import optimizers,initializers
import pandas as pd
from matplotlib import pyplot as plt
from keras.optimizers import RMSprop
from keras.utils import np_utils
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
sns.set(rc={'axes.facecolor':'white'})
pd.set_option('display.max_rows', 1000)

%matplotlib inline

/home/katya/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
def containsMutations(genotype):
    return genotype.split(':')

def makeBinary(unique_mutations, genotype):
    genotypeList = containsMutations(genotype)
    indexList = []
    
    for i in range(len(genotypeList)):
        indexList.append(unique_mutations.index(genotypeList[i]))
    
    line = np.zeros((1,len(unique_mutations)))
    line[:,indexList] = 1.
    
    return line

In [3]:
#############READING DATA#############
######EXTRACTING DATA AND LABELS######

def read_data(chunk):
    input_file = '~/Jupyter/HIS3InterspeciesEpistasis/Analysis/Katya/NN/data/' + chunk + '.txt'
    data = pd.read_table(input_file)
    data.columns = ['mutList', 'fitness', 'aa_seq']
    data.mutList = data.mutList.fillna('')
    unique_mutations = set(':'.join(data.mutList).split(':'))
    unique_mutations = sorted(list(unique_mutations))
    if '' in unique_mutations:
        unique_mutations.remove('')

#     data = data.reindex(np.random.permutation(data.index))
    nn_genotypes_values = np.zeros((len(data), len(unique_mutations)))
    nn_brightness_values = data.fitness.values
    aa_seq = data.aa_seq
    
    for i in range(len(data.mutList)):
        if data.mutList[i] != '':
            nn_genotypes_values[i] = makeBinary(unique_mutations, data.mutList[i])[0]
    
    return nn_genotypes_values, nn_brightness_values, unique_mutations, aa_seq

In [4]:
def makeBinaryDoubles(combinations,genotype):
    mutList = containsMutations(genotype)
    indexList = []
    
    for i in range(len(combinations)):
        if combinations[i][0] in mutList and combinations[i][1] in mutList:
            indexList.append(i)
            
    line = np.zeros((1,len(combinations)))
    line[:,indexList] = 1.
    
    return line   

In [5]:
#############READING DATA#############
######EXTRACTING DATA AND LABELS######
####features are double mutations####

def read_data_doubles(chunk):
    input_file = '~/Jupyter/HIS3InterspeciesEpistasis/Analysis/Katya/NN/data/' + chunk + '.txt'
    data = pd.read_table(input_file)
    data.columns = ['mutList', 'fitness', 'aa_seq']
    data.mutList = data.mutList.fillna('')
    unique_mutations = set(':'.join(data.mutList).split(':'))
    unique_mutations = sorted(list(unique_mutations))
    if '' in unique_mutations:
        unique_mutations.remove('')

    combinations = []

    for sub in itertools.combinations(unique_mutations, 2):
        combinations.append(sub)        
        
    data = data.reindex(np.random.permutation(data.index))
    nn_genotypes_values = np.zeros((len(data), len(combinations)))
    nn_brightness_values = data.fitness.values
        
    for i in range(len(data.mutList)):
        if data.mutList[i] != '':
            nn_genotypes_values[i] = makeBinaryDoubles(combinations, data.mutList[i])[0]

    nn_brightness_values = (nn_brightness_values - min(nn_brightness_values)) / max(
        nn_brightness_values - min(nn_brightness_values))

#     nn_genotypes_values = nn_genotypes_values[:, nn_genotypes_values.sum(axis=0) != 0]
    
    return nn_genotypes_values, nn_brightness_values, combinations

In [6]:
def makeBinaryTriples(combinations, genotype):
    mutList = containsMutations(genotype)
    indexList = []
    
    for i in range(len(combinations)):
        if combinations[i][0] in mutList and combinations[i][1] in mutList and combinations[i][2] in mutList:
            indexList.append(i)
            
    line = np.zeros((1,len(combinations)))
    line[:,indexList] = 1.
    
    return line   

In [7]:
#############READING DATA#############
######EXTRACTING DATA AND LABELS######
####features are triple mutations####

def read_data_triples(chunk):
    input_file = '~/Jupyter/HIS3InterspeciesEpistasis/Analysis/Katya/NN/data/' + chunk + '.txt'
    data = pd.read_table(input_file)
    data.columns = ['mutList', 'fitness', 'aa_seq']
    data.mutList = data.mutList.fillna('')
    unique_mutations = set(':'.join(data.mutList).split(':'))
    unique_mutations = sorted(list(unique_mutations))
    if '' in unique_mutations:
        unique_mutations.remove('')
    
    combinations = []

    for sub in itertools.combinations(unique_mutations, 3):
        combinations.append(sub)
    
    data = data.reindex(np.random.permutation(data.index))
    nn_genotypes_values = np.zeros((len(data), len(combinations)))
    nn_brightness_values = data.fitness.values
        
    for i in range(len(data.mutList)):
        if data.mutList[i] != '':
            nn_genotypes_values[i] = makeBinaryTriples(combinations, data.mutList[i])[0]

    nn_brightness_values = (nn_brightness_values - min(nn_brightness_values)) / max(
        nn_brightness_values - min(nn_brightness_values))
    
    nn_genotypes_values = nn_genotypes_values[:, nn_genotypes_values.sum(axis=0) != 0]
    
    return nn_genotypes_values, nn_brightness_values

In [8]:
# Plot the observed values versus predicted using density plot
def density_plot(x, y, chunk):
    ''' x = observed, y = predicted '''
    x = x[(~np.isnan(x)) & (~np.isnan(y))]
    y = y[(~np.isnan(x)) & (~np.isnan(y))]

    # Calculate the point density
    xy = np.vstack([x, y])
    z = gaussian_kde(xy)(xy)

    # Sort the points by density, so that the densest points are plotted last
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]

    # formatting
#     plt.figure(figsize=[8, 6])
    plt.scatter(x, y, c=z, s=3, edgecolor='', cmap='viridis_r')
#     plt.xlim(0, 1)
#     plt.ylim(0, 1)
    plt.xlabel('Observed brightness')
    plt.ylabel('Predicted brightness')
    plt.title(chunk)
    plt.tick_params(axis="both", which="both", bottom="off", top="off",
                    labelbottom="on", left="off", right="off", labelleft="on")
    plt.gca().spines["top"].set_visible(False)
    plt.gca().spines["right"].set_visible(False)
    plt.gca().spines["bottom"].set_color('gray')
    plt.gca().spines["left"].set_color('gray')
    plt.gca().xaxis.grid(True)
    plt.gca().yaxis.grid(True)

In [9]:
#############PLOTTING FITNESS POTENTIAL VS FITNESS#############

def fitness_potential_plotting(weights_set, num=1):
    plt.figure(figsize=(4*5+3,3*5))
    plt.suptitle('Fitness potential vs Fitness', size=30, x=0.8, y=1.6)
    count=1
    for chunk in chunks:
        plt.subplot(3,4,count)
        plt.subplots_adjust(top = 1.5,right=1.5)
        plt.title(chunk, fontsize=20)

        plt.plot(fitness_potential[chunk][:,weights_set][:10000], true[chunk][:10000], 'ok', alpha = 0.01)
        
        plt.plot(fitness_potential[chunk][:,weights_set][:10000], predicted[chunk][:10000], '.', c='#36D1C4', alpha = 0.03)
        
        if num == 2:
            plt.plot(fitness_potential[chunk][:,weights_set+1][:10000], predicted[chunk][:10000], '.', c='#FF006C', alpha = 0.01)

        plt.grid(True, ls='--', lw=0.5, dash_capstyle = 'round', c='gray')
        plt.xlabel('Fitness potential', fontsize=15)
        plt.ylabel('Observed values', fontsize=15)
        count+=1

In [10]:
chunks = [('S'+str(x)) for x in range(1,13)]

In [11]:
#############EXTRACTING UNIQUE MUTATIONS FOR EACH OF THE SEGMENTS#############

unique_mutations = {}
for chunk in chunks:
    input_file = '~/Jupyter/HIS3InterspeciesEpistasis/Analysis/Katya/NN/data/' + chunk + '.txt'
    df = pd.read_table(input_file)
    df.columns = ['mutList', 'fitness', 'aa_seq']
    df.mutList = df.mutList.fillna('')
    unique_mutations[chunk] = set(':'.join(df.mutList).split(':'))
    unique_mutations[chunk] = sorted(list(unique_mutations[chunk] ))
    if '' in unique_mutations[chunk]:
        unique_mutations[chunk] .remove('')

In [12]:
def disectSeq(sq):
    aaList = []
    counter = 0
    for a in sq:
        aaList.append(str(counter)+a)
        counter+=1
    return ':'.join(aaList)

In [13]:
def read_data_all_positions(chunk):
    
    input_file = '~/Jupyter/HIS3InterspeciesEpistasis/Analysis/Katya/NN/data/' + chunk + '.txt'
    data = pd.read_table(input_file)
    data.columns = ['mutList', 'fitness', 'aa_seq']
    
    mut_list = list(data.mutList)
    genotypeList = [disectSeq(sq) for sq in data.aa_seq]
    data.mutList += ':'
    data.mutList += pd.Series(genotypeList)
    
    data.mutList = data.mutList.fillna('')
    
    unique_mutations = set(':'.join(data.mutList).split(':'))
    unique_mutations = sorted(list(unique_mutations))
    if '' in unique_mutations:
        unique_mutations.remove('')

#     data = data.reindex(np.random.permutation(data.index))
    nn_genotypes_values = np.zeros((len(data), len(unique_mutations)))
    nn_brightness_values = data.fitness.values
    aa_seq = data.aa_seq
    
    for i in range(len(data.mutList)):
        if data.mutList[i] != '':
            nn_genotypes_values[i] = makeBinary(unique_mutations, data.mutList[i])[0]
    
    return nn_genotypes_values, nn_brightness_values, unique_mutations, aa_seq, data.mutList